In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from tqdm.notebook import tqdm
from consts import *
tqdm.pandas()


EMBED_FILE = get_filename(TT.CAP, EMBED_PROVIDER.OPENAI)
EMBED_FOUT = get_filename(TT.CAP, EMBED_PROVIDER.OPENAI, MEAN_METHOD.CENTROID)


df = pd.read_parquet(EMBED_FILE)
df

,username,caption,embeds
0,deparmedya,Cumhuriyetimizin 100.yılı kutlu olsun♾️🇹🇷,"[0.006900848, -0.0041164574, 0.003734105, 0.03..."
1,deparmedya,Oriflame Duologi Lansmanı #isveçtengelengüzell...,"[-0.027352218, -0.012979045000000002, -0.01375..."
2,deparmedya,#oriflameilesaçbakımdevrimi ✌️,"[-0.022362228, 0.00024338660000000002, -0.0006..."
3,deparmedya,✌️#oriflameilesaçbakımdevrimi 07Agustos’23 ori...,"[-0.027011871000000003, -0.011663851000000001,..."
4,deparmedya,07 Agustos’23 #oriflameturkiye #duoloji,"[-0.016240971, -0.013400658000000001, -0.00667..."
...,...,...,...
111112,tariszeytinyagi,"9 Eylül güzel İzmir’imizin kurtuluş günü, Cumh...","[0.020275634, -0.004595624, -0.004954088000000..."
111113,tariszeytinyagi,Tariş Zeytinyağı sofralarımıza uzanan şifa ve ...,"[0.006702404, 0.027326606000000003, -0.0025326..."
111114,tariszeytinyagi,Kendine özgün tadı ile damağınızda derin bir l...,"[0.022397906000000002, 0.007563821500000001, -..."
111115,tariszeytinyagi,"Serinleten Ayran Çorbası\nMalzemeler:\n1,5 su ...","[-0.0052431915, -0.0050309882, -0.0072267037, ..."


In [2]:
def combine_embeddings_with_relevance(embeddings):
    embeddings = np.array(embeddings)
    
    center_embedding = np.mean(embeddings, axis=0)
    
    def cosine_similarity(a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    similarities = np.array([cosine_similarity(embed, center_embedding) for embed in embeddings])
    
    weights = similarities - np.min(similarities)
    weights /= np.sum(weights)
    
    weighted_embedding = np.sum(embeddings.T * weights, axis=1)
    
    return weighted_embedding

In [4]:
usernames = df["username"].unique().tolist()

new_df = []

for username in tqdm(usernames):
    rows = df[df["username"] == username]
    embeds = []
    for index, row in rows.iterrows():
        embeds.append(row["embeds"])

    mean_embed = combine_embeddings_with_relevance(embeds)

    new_df.append((username, mean_embed))

  0%|          | 0/3470 [00:00<?, ?it/s]

/tmp/ipykernel_661001/3869154417.py:12: RuntimeWarning: invalid value encountered in divide
  weights /= np.sum(weights)


In [5]:

new = pd.DataFrame(new_df)
new.columns = ['username','embeds']
new

,username,embeds
0,deparmedya,"[-0.016066564151752914, -0.0017462519867390868..."
1,kafesfirin,"[-0.012462408379125598, -0.0036353223014306667..."
2,vimerang,"[-0.01224335002665966, 0.0037703351268183255, ..."
3,mustafa_yalcinn38,"[0.0025654376214577527, 0.004133783181529795, ..."
4,drtubagunebak,"[0.010873177619945974, 0.0007548815059353719, ..."
...,...,...
3465,woodtechistanbul,"[-0.017239120774057266, 0.022620527848517032, ..."
3466,netsvadi,"[-0.011835124259297062, 0.007105485990351733, ..."
3467,herbisiatolye,"[0.0076287485043709325, 0.007663077453733879, ..."
3468,nestleturkiye,"[0.0005994182932900033, -0.005860620021275915,..."


In [6]:
new.to_parquet(EMBED_FOUT)